Ensure you are using the preinstalled Jupyter kernel `amcinsights`

# Amazon Ads Authorization
The steps in this notebook only need to be followed once in order to authorize your client credentials and store your access tokens. If your client information changes or access is revoked, come back and perform these steps once again.

### Before running the code in this notebook complete the following steps:

1. Create an [authorization grant](https://advertising.amazon.com/API/docs/en-us/guides/get-started/create-authorization-grant) and set https://amazon.com as an Allowed Return URL

**NOTE:** the following steps must be completed within 5 minutes as the authorization code will only be valid for this time. If the Lambda function is not triggered within this timeframe you will have to repeat steps 2 - 5

2. Replace CLIENT_ID with your client id value and navigate to the URL: `https://www.amazon.com/ap/oa?client_id=CLIENT_ID&scope=profile%20advertising::campaign_management&response_type=code&redirect_uri=https://amazon.com`

3. Copy your authorization code from the opened browser redirect page (ex. for the code XXXXXXXXXX: `https://www.amazon.com/?code=XXXXXXXXXXscope=profile%20advertising::campaign_management`)

4. Store your authorization code, client id, and client secret in Secrets Manager in the wfmSecret. A link to this secret can be found in your Cloudformation stacks Output tab under the key `wfmAMCSecrets946833FA`. [Click here](https://docs.aws.amazon.com/secretsmanager/latest/userguide/manage_update-secret-value.html) for instructions on how to update this value. If managing multiple credentials, refer to the [Implementation Guide](https://docs.aws.amazon.com/solutions/latest/amazon-marketing-cloud-insights-on-aws/supplemental-topics.html#managing-multiple-authenticated-credentials) for how to structure your secret.

5. Trigger the Amazon Ads Auth Lambda function by running the code below 

For more information please refer to the [official documentation](https://advertising.amazon.com/API/docs/en-us/guides/get-started/overview)

In [ ]:
import boto3
import json
import os
from dotenv import load_dotenv
load_dotenv()

amazon_ads_auth_lambda_name = os.environ['AMAZON_ADS_AUTH_LAMBDA_NAME']

boto3_session = boto3.Session()
lambda_client = boto3_session.client('lambda')

# (Optional) If multi-credential secret configured, Auth Id to use for for authenticating
# If you did not configure your secret for multiple credentials, leave as None.
auth_request = {
    "auth_id" : None
}

response = lambda_client.invoke(
    FunctionName=amazon_ads_auth_lambda_name,
    InvocationType='RequestResponse',
    LogType='Tail',
    Payload=json.dumps(auth_request).encode('UTF-8'),
)

response_code = response.get('ResponseMetadata', {}).get('HTTPStatusCode', 0)
if response_code in range(200, 204):
    function_url = f"https://{os.environ['REGION']}.console.aws.amazon.com/lambda/home?region={os.environ['REGION']}#/functions/{amazon_ads_auth_lambda_name}"
    print(f"Function URL {function_url}")
else:
    print(response)

----
Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.

SPDX-License-Identifier: Apache-2.0